# `ipython`
> Set of utility functions to be used in Jupyter and Jupyter Lab notebooks.


In [ ]:
#|default_exp ipython

In [ ]:
#| export
from __future__ import annotations
from fastcore.test import test_fail
from functools import wraps
from IPython.core.getipython import get_ipython
from IPython.display import display, Markdown, display_markdown
from pathlib import Path
from typing import Any, List, Callable, Optional
from eccore.core import safe_path, path_to_parent_dir, is_type, CurrentMachine

import numpy as np
import pandas as pd
import sys
import subprocess
import warnings

In [ ]:
#| hide
from nbdev import show_doc, nbdev_export

# System and CLI

In [ ]:
#| export

# TODO: replace with fastcore run

def run_cli(cmd:str = 'ls -l'   # command to execute in the cli
           ):
    """Runs a cli command from jupyter notebook and print the shell output message
    
    Uses subprocess.run with passed command to run the cli command"""
    p = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
    print(str(p.stdout, 'utf-8'))

In [ ]:
run_cli('pwd')

/home/vtec/projects/ec-packages/eccore/nbs-dev



# Notebook setup

In [ ]:
#| export

# TODO: update using the ipython functions of fastcore

def nb_setup(
    autoreload:bool = True,       # True to set autoreload in this notebook
    paths:List[str|Path] = None   # Paths to add to the path environment variable
    ):
    """Use in first cell of notebook to set autoreload, and add system paths
    
    Always add a path to the directoruy 'src' if `srs` directory exists at the same level as the `nbs` directory.

    When the notebook is not located in a tree including the name `nbs`, `src` directory is searched at the same level
    as the directory in which the notebook is located.
    """
    #  Handle paths
    #  Add src if it exists
    nbs_root = path_to_parent_dir('nbs')
    p2src = (nbs_root / '../src').resolve().absolute()
    if p2src.is_dir():
        p = str(p2src.absolute())
        if p not in sys.path:
            sys.path.insert(0, p)
            print(f"Added path: {p2src.absolute()}")
    # Add passed paths
    if paths:
        for p in paths:
            if isinstance(p, Path): p = str(p.resolve().absolute())
            if p not in sys.path:
                sys.path.insert(1, p)
                print(f"Added path: {p}")

#   Setup auto reload
    if autoreload:
        ipshell = get_ipython()
        ipshell.run_line_magic('load_ext',  'autoreload')
        ipshell.run_line_magic('autoreload', '2')
        print('Set autoreload mode')

In [ ]:
show_doc(nb_setup)

---

### nb_setup

>      nb_setup (autoreload:bool=True, paths:List[str|pathlib.Path]=None)

Use in first cell of notebook to set autoreload, and add system paths

Always add a path to the directoruy 'src' if `srs` directory exists at the same level as the `nbs` directory.

When the notebook is not located in a tree including the name `nbs`, `src` directory is searched at the same level
as the directory in which the notebook is located.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| autoreload | bool | True | True to set autoreload in this notebook |
| paths | List[str \| Path] | None | Paths to add to the path environment variable |

By default, `ipython.nb_setup()` 
- loads and set `autoreload`
- adds a path to a directory named `src` when it exists at the same level as where the notebook directory is located. It no such `src` directory exists, no path is added

`ipython.nb_setup` assumes the following file structure:

```
    project_directory
          |--- nbs
          |     | --- current_nb.ipynb
          |     | --- ...
          |
          |--- src
          |     | --- module_to_import.py
          |     | --- ...
          |
          |--- data
          |     |
          |     | ...
```

For other file structure, specify paths as a `list` of `Path`

Before running `nb_setup`, `sys.path` does not include the path to the local source directory. After running it, it will be added, unless the directory does not exist.

In [ ]:
sys.path

['/home/vtec/projects/ec-packages/eccore/nbs-dev',
 '/home/vtec/projects/lewagon/1286-Jul-2023/1_data-challenges/data-context-and-setup',
 '/home/vtec/projects/ec-packages/eccore/nbs-dev',
 '/home/vtec/miniconda3/envs/eccore/lib/python310.zip',
 '/home/vtec/miniconda3/envs/eccore/lib/python3.10',
 '/home/vtec/miniconda3/envs/eccore/lib/python3.10/lib-dynload',
 '',
 '/home/vtec/miniconda3/envs/eccore/lib/python3.10/site-packages',
 '/home/vtec/miniconda3/envs/eccore/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg',
 '/home/vtec/projects/ec-packages/eccore']

In [ ]:
nb_setup(autoreload=False)

In [ ]:
sys.path

['/home/vtec/projects/ec-packages/eccore/nbs-dev',
 '/home/vtec/projects/lewagon/1286-Jul-2023/1_data-challenges/data-context-and-setup',
 '/home/vtec/projects/ec-packages/eccore/nbs-dev',
 '/home/vtec/miniconda3/envs/eccore/lib/python310.zip',
 '/home/vtec/miniconda3/envs/eccore/lib/python3.10',
 '/home/vtec/miniconda3/envs/eccore/lib/python3.10/lib-dynload',
 '',
 '/home/vtec/miniconda3/envs/eccore/lib/python3.10/site-packages',
 '/home/vtec/miniconda3/envs/eccore/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg',
 '/home/vtec/projects/ec-packages/eccore']

We also can add other specific paths:

In [ ]:
path_to_add = str(Path('../nbs').resolve().absolute())
nb_setup(autoreload=False, paths=[path_to_add])

Added path: /home/vtec/projects/ec-packages/eccore/nbs


In [ ]:
sys.path

['/home/vtec/projects/ec-packages/eccore/nbs-dev',
 '/home/vtec/projects/ec-packages/eccore/nbs',
 '/home/vtec/projects/lewagon/1286-Jul-2023/1_data-challenges/data-context-and-setup',
 '/home/vtec/projects/ec-packages/eccore/nbs-dev',
 '/home/vtec/miniconda3/envs/eccore/lib/python310.zip',
 '/home/vtec/miniconda3/envs/eccore/lib/python3.10',
 '/home/vtec/miniconda3/envs/eccore/lib/python3.10/lib-dynload',
 '',
 '/home/vtec/miniconda3/envs/eccore/lib/python3.10/site-packages',
 '/home/vtec/miniconda3/envs/eccore/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg',
 '/home/vtec/projects/ec-packages/eccore']

In [ ]:
#| export
def install_code_on_cloud(
    package_name:str, # project package name, e.g. metagentools or git+https://github.com/repo.git@main
    quiet:bool=False # install quietly with Trud
):
    """pip install the project code package, when nb is running in the cloud."""
    
    machine = CurrentMachine()

    if machine.is_colab:
        CLOUD = True
        print('The notebook is running on colab.', end=' ')
        print(f'Will install {package_name}')
    elif machine.is_kaggle:
        CLOUD = True
        print('The notebook is running on kaggle.', end=' ')
        print(f'Will install {package_name}')
    elif machine.is_local:
        CLOUD = False
        print('The notebook is running locally, will not automatically install project code')
    else:
        CLOUD = True
        print('The notebook is running on a cloud VM or the machine was not registered as local')
        print(f'Will install {package_name}')

    if CLOUD:
        print(f'Installing project code {package_name}')
        cmd = f"pip install -{'qq' if quiet else ''}U {package_name}"
        run_cli(cmd)
        print((f"{package_name} is installed."))

In [ ]:
show_doc(install_code_on_cloud)

---

### install_code_on_cloud

>      install_code_on_cloud (package_name:str, quiet:bool=False)

pip install the project code package, when nb is running in the cloud.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| package_name | str |  | project package name, e.g. metagentools or git+https://github.com/repo.git@main |
| quiet | bool | False | install quietly with Trud |

When using colab, kaggle or another cloud VM, specicif code must be installed every time from the Python Package Index (PyPI) or its GitHub repo.

When running locally, the project code should be pre-installed as part of the environment

In [ ]:
install_code_on_cloud(package_name='metagentools');

The notebook is running locally, will not automatically install project code


# Improve output cell formats

In [ ]:
#| export
def display_mds(
    *strings:str|tuple[str] # any number of strings with text in markdown format
):
    """Display one or several strings formatted in markdown format"""
    for string in strings:
        display_markdown(Markdown(data=string))

In [ ]:
show_doc(display_mds)

---

### display_mds

>      display_mds (*strings:str|tuple[str])

Display one or several strings formatted in markdown format

In [ ]:
display_mds('**bold** and _italic_')

**bold** and _italic_

In [ ]:
display_mds('**bold** and _italic_',
            '- bullet',
            '- bullet',
            '> Note: this is a note'
)

**bold** and _italic_

- bullet

- bullet

> Note: this is a note

In [ ]:
#| export
def display_dfs(*dfs:pd.DataFrame       # any number of Pandas DataFrames
               ):
    """Display one or several `pd.DataFrame` in a single cell output"""
    for df in dfs:
        display(df)

In [ ]:
show_doc(display_dfs)

---

### display_dfs

>      display_dfs (*dfs:pandas.core.frame.DataFrame)

Display one or several `pd.DataFrame` in a single cell output

In [ ]:
df1 = pd.DataFrame(data=np.random.normal(size=(10,5)))
df2 = pd.DataFrame(data=np.random.normal(size=(20,10)))

display_dfs(df1.head(3), df2.head(3))

,0,1,2,3,4
0,-2.142192,1.164811,-1.089103,-0.451463,-0.553741
1,-0.104041,-0.378835,0.158306,0.856799,-0.868164
2,0.152312,1.427305,0.220376,-1.602171,-0.296513


,0,1,2,3,4,5,6,7,8,9
0,-0.970582,0.742558,0.456552,-1.173472,-1.044596,0.928496,1.082050,0.134465,1.366026,-0.543924
1,0.310118,1.779701,-0.721666,-1.453407,0.635389,-0.053451,-1.327018,-2.173942,1.155529,-0.077860
2,-1.184941,0.502554,-1.317360,-0.660704,-0.357762,-0.711923,0.291072,-1.144267,1.150530,0.366801


In [ ]:
#| export
class pandas_nrows_ncols:
    """Context manager that sets the max number of rows and cols to apply to any output within the context"""
    def __init__(
        self, 
        nrows:int|None=None, # max number of rows to show; show all rows if `None`
        ncols:int|None=None, # max number of columns to show; show all columns if `None`
    ):
        self.nrows = nrows
        self.ncols = ncols
    
    def __enter__(self):
        self.max_rows = pd.options.display.max_rows
        self.max_cols = pd.options.display.max_columns
        pd.options.display.max_rows = self.nrows
        pd.options.display.max_columns = self.ncols
        return self.max_rows, self.max_cols

    def __exit__(self, exc_type, exc_value, exc_tb):
        pd.options.display.max_rows = self.max_rows
        pd.options.display.max_columns = self.max_cols

In [ ]:
show_doc(pandas_nrows_ncols)

---

### pandas_nrows_ncols

>      pandas_nrows_ncols (nrows:int|None=None, ncols:int|None=None)

Context manager that sets the max number of rows and cols to apply to any output within the context

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| nrows | int \| None | None | max number of rows to show; show all rows if `None` |
| ncols | int \| None | None | max number of columns to show; show all columns if `None` |

With no context manager, the pandas object are displayed with a maximum of 60 rows and 20 columns.

In [ ]:
df = pd.DataFrame(np.random.randint(low=0, high=100, size=(3,50)))
display(df)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,78,44,12,74,69,78,37,7,2,74,...,67,85,57,80,70,18,46,99,42,11
1,7,64,71,43,86,50,61,63,44,21,...,73,66,38,76,15,45,13,42,23,26
2,36,80,36,19,89,25,91,63,91,26,...,70,24,80,27,91,78,88,18,78,89


Using the context manager, all rows and columns will be displayed

In [ ]:
with pandas_nrows_ncols():
    display(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,78,44,12,74,69,78,37,7,2,74,64,9,51,16,87,97,5,20,99,98,9,34,66,29,6,53,43,32,81,42,79,57,93,95,95,91,31,32,96,44,67,85,57,80,70,18,46,99,42,11
1,7,64,71,43,86,50,61,63,44,21,70,68,38,3,44,21,8,97,52,31,94,33,0,99,11,63,78,74,89,54,7,57,88,21,13,58,94,15,53,36,73,66,38,76,15,45,13,42,23,26
2,36,80,36,19,89,25,91,63,91,26,39,79,80,54,26,83,74,4,70,61,49,16,95,31,14,10,85,94,7,6,39,9,83,30,69,47,46,93,81,21,70,24,80,27,91,78,88,18,78,89


It is also possible to specifically define the number of rows and columns to display

In [ ]:
with pandas_nrows_ncols(nrows=2, ncols=6):
    display(df)

,0,1,2,...,47,48,49
0,78,44,12,...,99,42,11
...,...,...,...,...,...,...,...
2,36,80,36,...,18,78,89


In [ ]:
with pandas_nrows_ncols(2,6):
    print(df)

    0   1   2   ...  47  48  49
0   78  44  12  ...  99  42  11
..  ..  ..  ..  ...  ..  ..  ..
2   36  80  36  ...  18  78  89

[3 rows x 50 columns]


> ### Technical background
> 
> the context manager uses pandas's [`options API`](https://pandas.pydata.org/docs/user_guide/options.html)

In [ ]:
pd.options.display.max_rows, pd.options.display.max_columns

(60, 20)

In [ ]:
pd.get_option('display.max_rows'), pd.get_option('display.max_columns')

(60, 20)

In [ ]:
pd.describe_option('display.max_rows')

display.max_rows : int
    If max_rows is exceeded, switch to truncate view. Depending on
    `large_repr`, objects are either centrally truncated or printed as
    a summary view. 'None' value means unlimited.

    In case python/IPython is running in a terminal and `large_repr`
    equals 'truncate' this can be set to 0 and pandas will auto-detect
    the height of the terminal and print a truncated object which fits
    the screen height. The IPython notebook, IPython qtconsole, or
    IDLE do not run in a terminal and hence it is not possible to do
    correct auto-detection.
    [default: 60] [currently: 60]


In [ ]:
pd.options.display.max_rows = 10
pd.reset_option('display.max_rows')
pd.options.display.max_rows

60

In [ ]:
#| export
def display_full_df(
    df:pd.DataFrame|pd.Series,  # `DataFrame` or `Series` to display
):
    """Display a pandas `DataFrame` or `Series` showing all rows and columns"""
    if is_type(df, pd.DataFrame, raise_error=False) or is_type(df, pd.Series, raise_error=False):
        with pandas_nrows_ncols():
            display(df)
    else:
        raise TypeError(f"df must me a pandas `DataFrame` or `Series`, not a {type(df)}")

In [ ]:
show_doc(display_full_df)

---

### display_full_df

>      display_full_df
>                       (df:pandas.core.frame.DataFrame|pandas.core.series.Serie
>                       s)

Display a pandas `DataFrame` or `Series` showing all rows and columns

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| df | pd.DataFrame \| pd.Series | `DataFrame` or `Series` to display |

In [ ]:
df = pd.DataFrame(np.random.randint(low=0, high=100, size=(3,50)))
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,29,48,36,90,34,78,62,78,18,71,...,79,45,87,2,59,72,96,47,22,42
1,39,96,37,91,3,59,78,9,16,50,...,61,6,87,45,28,4,40,27,96,61
2,72,65,87,73,3,86,21,81,89,34,...,77,45,62,53,63,20,77,5,90,72


In [ ]:
display_full_df(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,29,48,36,90,34,78,62,78,18,71,87,73,94,63,26,0,21,10,27,48,41,99,38,68,72,74,8,41,29,66,85,80,86,15,87,89,57,44,50,86,79,45,87,2,59,72,96,47,22,42
1,39,96,37,91,3,59,78,9,16,50,49,39,17,67,73,86,69,67,90,14,67,54,12,9,64,49,12,45,44,47,56,21,13,1,46,43,98,55,88,49,61,6,87,45,28,4,40,27,96,61
2,72,65,87,73,3,86,21,81,89,34,93,14,42,35,47,20,29,41,11,19,62,12,19,48,64,79,44,32,62,13,34,19,48,25,55,25,3,48,14,14,77,45,62,53,63,20,77,5,90,72


In [ ]:
#| hide
display_full_df(df.loc[0, :].T)

0     29
1     48
2     36
3     90
4     34
5     78
6     62
7     78
8     18
9     71
10    87
11    73
12    94
13    63
14    26
15     0
16    21
17    10
18    27
19    48
20    41
21    99
22    38
23    68
24    72
25    74
26     8
27    41
28    29
29    66
30    85
31    80
32    86
33    15
34    87
35    89
36    57
37    44
38    50
39    86
40    79
41    45
42    87
43     2
44    59
45    72
46    96
47    47
48    22
49    42
Name: 0, dtype: int64

In [ ]:
msg = 'should raise a TypeError'
contains = 'df must me a pandas `DataFrame` or `Series`'

test_fail(display_full_df, args=['a string'], msg=msg, contains=contains)

In [ ]:
#| hide
nbdev_export()